## Clase 5 - Ejercicios para la clase teórica

### Descargar archivos

In [ ]:
import requests
import pandas as pd

Similar a la función que hicimos al comienzo de la clase dos (proyecto simulación contagios COVID-19) a continuacion creamos una funcion que solicita el contenido de un archivo en la web y lo escribe en un documento csv. 

In [ ]:
def descarga_base_infectados(csv_url, csv_name):
    '''
    Esta funcion solicita el contenido de un archivo en la web y lo escribe en 
    un documento csv. 
    
    Input:
        csv_url (str): url de donde descargar el archivo
        csv_name (str): nombre con el cual guardar el archivo descargado
    '''
    
    #Obtenemos el contenigo del archivo en el url especificado
    req = requests.get(csv_url)
    url_content = req.content

    #Guardamos el contenido descargado en un archivo con el nombre que elegimos
    csv_file = open(csv_name, 'w')
    csv_file.write(url_content.decode())
    csv_file.close()

In [ ]:
#Definimos constantes que usaremos de ejemplo en esta función

#La base de infectados de COVID es bastante grande hoy en día. Puede tardar 
#algunos minutos en descargar. Es por ello que en el TP2 les hemos dado las 
#bases ya descargadas. En este ejemplo, si les tarda mucho en correr prueben 
#con algún otro archivo de la web que les interese.
CSV_URL = "https://cdn.buenosaires.gob.ar/datosabiertos/datasets/salud/casos-covid-19/casos_covid19.csv"
CSV_NAME = "casos_covid19.csv"

In [ ]:
descarga_base_infectados(CSV_URL, CSV_NAME)

Para probar si el archivo se descargó podemos ir a la carpeta donde tenemos este jupyter notebook y verificar si existe un archivo llamado "casos_covid19.csv" y tiene contenido. Otra opción es importar el contenido desde el csv como un dataframe:

In [ ]:
df = pd.read_csv(CSV_NAME)
df.head

### Web Scraping

In [ ]:
import requests #html requestor
from bs4 import BeautifulSoup #html parser
import pandas as pd #dataframe manipulator

In [ ]:
url = 'https://datatables.net/examples/basic_init/zero_configuration.html'

Solicitamos el html del url indicado. El codigo de respuesta 200 significa que la respuesta del sitio fue exitosa

In [ ]:
response = requests.get(url)
print(response.status_code)

Dividimos el texto con `BeautifulSoup`.

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')
print(soup)

Se puede observar que la informacion que queremos extraer esta entre las etiquetas llamadas `tr`. Por lo tanto queremos encontrar todas las etiquetas `tr`.

In [ ]:
info = soup.find_all('tr')
info

In [ ]:
print(type(info))
print(type(info[0]))

Podemos ver que todas las observaciones, excepto la primera y la última, contienen la información que necesitamos. También observamos que el nombre, el puesto, el cargo, la edad, la fecha de inicio y el salario siempre tienen el mismo orden. Podemos hacer uso de estos patrones para extraer la información en un marco de datos.

In [ ]:
df = pd.DataFrame(columns=['name', 'position', 'office', 'age', 'start date', 'salary'])

In [ ]:
for i, item in enumerate(info): 
    if i != 0 and i != len(info)-1:
        datos_de_fila = item.find_all('td')
        fila = []
        for dato in datos_de_fila:
            fila.append(dato.text)
        print('\nFila:', i)
        print(fila)
        df.loc[i-1] = fila 


In [ ]:
display(df)

Por último exportamos la información a un archivo `csv`:

In [ ]:
df.to_csv('final_output.csv', index=False)

### PDF scraping

In [ ]:
import pandas as pd
import PyPDF2
import tabula

Usando PyPDF2 extraigo todo el texto del PDF

In [ ]:
pdf_file = open('boletin_ife.pdf', 'rb')
pdfReader = PyPDF2.PdfFileReader(pdf_file)
paginas = pdfReader.numPages
for i in range(paginas):
    print("Este es el contenido de la pagina {}:".format(i+1))
    pagina = pdfReader.getPage(i)
    print(pagina.extractText())

Usando tabula puedo extraer las tablas del PDF

In [ ]:
lst_of_df = tabula.read_pdf('boletin_ife.pdf', output_format='dataframe', pages=7)

In [ ]:
lst_of_df

In [ ]:
type(lst_of_df)

In [ ]:
type(lst_of_df[0]) 

In [ ]:
tabla_p7 = lst_of_df[0]
tabla_p7.columns = ['tipo_de_hogar','pobreza', 'indigencia']
tabla_p7

In [ ]:
lst_of_df = tabula.read_pdf('boletin_ife.pdf', output_format='dataframe', pages=30)
lst_of_df

In [ ]:
#Tabulas measures: x inches * 72
header = ["provincia",
         "18_a_24_anos", 
         "25_a_34_anos",
         "35_a_44_anos",
         "45_a_54_anos",
         "55_a_65_anos",
         "total"]
perimetro = (2.35*72, 1.19*72, 7*72, 7.07*72)
columnas = (2.5*72, 3.28*72, 4.04*72, 4.79*72, 5.58*72, 6.30*72)
lst_of_df = tabula.read_pdf('boletin_ife.pdf', output_format='dataframe', 
                            pages=30, area = perimetro, columns = columnas)

tabla_p30 = lst_of_df[0]
tabla_p30.columns = header
tabla_p30

In [ ]:
lst_of_df = tabula.read_pdf('boletin_ife.pdf', output_format='dataframe', pages=33)
print(len(lst_of_df))


In [ ]:
lst_of_df

In [ ]:
lst_of_df[0].to_csv('pabla_pag33.csv', index=False)

### Bases de datos relacionales


In [ ]:
import pandas as pd
import sqlite3

In [ ]:
#Iniciamos la conexion a la base de datos con el metodo 'connect'
conn = sqlite3.connect('IFE.db')

#creamos un cursor a la base para poder comenzar a ejecuter los comandos de SQL:
c = conn.cursor()

# Crear tabla
c.execute('''CREATE TABLE provincias
             (provincia text, prov_cod INTEGER PRIMARY KEY)''')

# Insertar una linea de datos
c.execute("INSERT INTO provincias VALUES ('Buenos Aires', 1)")

# INsertar varios registros a la vez
provincias = [('CABA', 2),
             ('Catamarca', 3),
             ('Chaco', 4)]

c.executemany('INSERT INTO provincias VALUES (?,?)', provincias)

# Guardar (commit) los cambios
conn.commit()

#Una vez que hacen el comit, los cambios se guardan y estaran disponibles para
#la proxima vez que nos conectemos a la base. Aquellos cambios que hagamos pero 
#no guardemos se borraran al cerrar la conexion a la base de datos. 

#Cerrar la conexión:
conn.close()


In [ ]:
#Iniciamos la conexion a la base de datos con el metodo 'connect'
conn = sqlite3.connect('IFE.db')

#creamos un cursor a la base para poder comenzar a ejecuter los comandos de SQL:
c = conn.cursor()


In [ ]:
c.execute('SELECT * FROM provincias')
print(c.fetchall())

In [ ]:
c.execute('DROP TABLE IF EXISTS provincias;')


In [ ]:
c.execute('SELECT * FROM provincias')
print(c.fetchall())


In [ ]:
# Guardar (commit) los cambios
#conn.commit()

#Cerrar la conexión:
conn.close()

In [ ]:
lst_p30 = tabla_p30.to_numpy()
lst_p30 

In [ ]:
#Iniciamos la conexion a la base de datos con el metodo 'connect'
conn = sqlite3.connect('IFE.db')

#creamos un cursor a la base para poder comenzar a ejecuter los comandos de SQL:
c = conn.cursor()

# Crear tabla
c.execute('''CREATE TABLE hogares_uni
             (provincia text, anos18a24 int, anos25a34 int, 
             anos35a44 int, anos45a54 int, anos55a56 int, 
             total int)''')

c.executemany('INSERT INTO hogares_uni VALUES (?,?,?,?,?,?,?)', lst_p30)

# Guardar (commit) los cambios
conn.commit()

#Una vez que hacen el comit, los cambios se guardan y estaran disponibles para
#la proxima vez que nos conectemos a la base. Aquellos cambios que hagamos pero 
#no guardemos se borraran al cerrar la conexion a la base de datos. 

#Cerrar la conexión:
conn.close()


In [ ]:
conn = sqlite3.connect('IFE.db')

c = conn.cursor()

provincia = ("CABA",)
c.execute('SELECT total FROM hogares_uni WHERE provincia == ?', provincia)
print(c.fetchone())


c.execute('SELECT * FROM hogares_uni WHERE provincia == ?', provincia)
print(c.fetchall())

conn.close()